# The Tensor Class

Our list of tests to implement currently contains only one test:

- Christoffel symbols should be accessible by coordinate names

I would prefer to implement this feature later, because it is not super urgent. Generally it is better to concentrate on essential features first and do those which are nice to have later. This approach is called *vertical slicing* and makes sure that you quickly get customer value. With our Manifold class in place we can turn our attention to implementing the notion of a Tensor field. 

Tensors live on manifolds. So the tensor should know about its manifold. And since we have a coordinate system, we can express the tensor in terms of coordinate-specific components. For example, we could a rank rank three tensor accepting a vector, a one-form and another vector as input, the components in the coordinate bases would be

$$
A^{\mu}\,_{\nu}\,^\alpha
$$


with index positions *up-down-up*. These components should be stored in an appropriate data structure. 

The usual tensor algebra includes addition of tensors, tensor products, index raising and lowering, contraction and covariant differentiation. Let's start with the easy cases, which is just storing the values and addition of tensors. The tests that immediately come up my mind for this are:

- Initializing a tensor with values of components, stores these values.
- Initializing a tensor without values is the zero tensor.
- Adding two tensors of the same structure yields another tensor with the same structure, where the component values are the sum of the componenents of the single tensors
- Adding two tensors with different structure raises an expection.

#### Test: Initializing a tensor with values of components, stores these values.

Write a **new** class in `tests/test_diffgeom.py`:



```python

from diffgeom import Tensor

class TestTensor(unittest.TestCase):

    def test_init_tensor_with_values_stores_values(self):
        # Arrange
        r, phi = sp.symbols('r, phi')
        metric = sp.diag(1, r**2)
        plane = Manifold(metric, coords=(r, phi))

        # Act
        A = Tensor(plane, 'ulu', {(0, 1, 0): r**4, (1, 0, 1): 1/r**2})

        # Assert
        self.assertEqual(len(A), 2)
        self.assertEqual(A[0, 1, 0], r**4)
        self.assertEqual(A[1, 0, 1], 1 / r**2)

```

We create the tensor with the manifold on which it lives (`plane`), configure its structure (`ulu` for *up-low-up*) and give its values as a dictionary. The keys of dictionary are index tuples of the tensor components and nonzero values are taken here as $r^4$ and $1/r^2$ as an example. Also not how we then access the tensor components with the bracket notation.

Again, running all tests now will result in one failed test (the new one!). So let's turn to the implementation.

In `diffgeom/diffgeom.py` add a new class:

In [ ]:
class Tensor(object):

    def __init__(self, manifold, idx_pos, values):
        self.manifold = manifold
        self.idx_pos = idx_pos
        self.values = values

and add the name `Tensor` to the `diffgeom/__init__.py` file:

```python
from .diffgeom import Manifold, Tensor
```

Running the tests will yield an error complaining that objects of type Tensor don't have a `len()` property. So let's implement it:

```python
class Tensor(object):

    def __init__(self, manifold, idx_pos, values):
        self.manifold = manifold
        self.idx_pos = idx_pos
        self.values = values

    def __len__(self):
        return len(self.values)
```



Running the tests doesn't complain about the `len()` property any more, but says

`TypeError: 'Tensor' object is not subscriptable`

Ok, so we have to implement the bracket notation for the tensor class:

```python

class Tensor(object):

    def __init__(self, manifold, idx_pos, values):
        self.manifold = manifold
        self.idx_pos = idx_pos
        self.values = values

    def __len__(self):
        return len(self.values)

    def __getitem__(self, m_idx):
        return self.values[m_idx]
```

This makes all tests pass. So let's commit the code.

A comment about my notation: I write `m_idx` which stands for *multi-index* or index tuple.

The `__getitem__` method is called whenever we apply the bracket notation to a `Tensor` object in order to retrieve a component value. But what if we want to **set** a component value with the bracket notation? Well, let's write a test for that!

#### Test: Setting tensor component with bracket notation stores value.

Add a test to the `TestTensor` class in `tests/diffgeom.py`:

```python
    def test_set_tensor_component_stores_value(self):
        # Arrange
        r, phi = sp.symbols('r, phi')
        metric = sp.diag(1, r**2)
        plane = Manifold(metric, coords=(r, phi))
        A = Tensor(plane, 'ulu', {(0, 1, 0): r ** 4, (1, 0, 1): 1 / r ** 2})

        # Act
        A[1, 1, 1] = r**3

        # Assert
        self.assertEqual(A[1, 1, 1], r**3)
```

Running this test complains saying:

```
A[1, 1, 1] = r**3
TypeError: 'Tensor' object does not support item assignment
```

Ok, setting values with bracket notation does not work yet. We can fix that by adding an implementation for the `__setitem__` method:



```python
class Tensor(object):

    def __init__(self, manifold, idx_pos, values):
        self.manifold = manifold
        self.idx_pos = idx_pos
        self.values = values

    def __len__(self):
        return len(self.values)

    def __getitem__(self, m_idx):
        return self.values[m_idx]

    def __setitem__(self, m_idx, value):
        self.values[m_idx] = value

```

Running the tests should succeed now.

#### Test: Initializing a tensor without values is the zero tensor.

```python
    def test_init_tensor_without_values_is_zero_tensor(self):

        # Arrange
        r, phi = sp.symbols('r, phi')
        metric = sp.diag(1, r ** 2)
        plane = Manifold(metric, coords=(r, phi))
        # Act
        A = Tensor(plane, 'ulu')

        # Assert
        self.assertEqual(len(A), 0)

```

yields error

```
TypeError: __init__() missing 1 required positional argument: 'values'
```

fix

```python
class Tensor(object):

    def __init__(self, manifold, idx_pos, values=None):
        self.manifold = manifold
        self.idx_pos = idx_pos
        if values is None:
            self.values = {}
        else:
            self.values = values

    def __len__(self):
        return len(self.values)

    def __getitem__(self, m_idx):
        return self.values[m_idx]

    def __setitem__(self, m_idx, value):
        self.values[m_idx] = value
```

(the default value for `values`)

commit.

## Refactoring

Christoffels are not tensors but have similarity: they are objects with indices. Let's pull out the common behavior.

Introduce IndexedObject class and let Tensor inherit from it:

```python

class IndexedObject(object):

    def __init__(self, values=None):
        if values is None:
            self.values = {}
        else:
            self.values = values

    def __len__(self):
        return len(self.values)

    def __getitem__(self, m_idx):
        return self.values[m_idx]

    def __setitem__(self, m_idx, value):
        self.values[m_idx] = value


class Tensor(IndexedObject):

    def __init__(self, manifold, idx_pos, values=None):
        super(Tensor, self).__init__(values)
        self.manifold = manifold
        self.idx_pos = idx_pos
```

Running tests should still work.

Define class Christoffel:

```python

class Manifold(object):

    def __init__(self, metric, coords):
        self.metric = metric
        self.coords = coords
        self.gammas = Christoffel(coords, metric)



class Christoffel(IndexedObject):

    def __init__(self, coords, metric):
        super(Christoffel, self).__init__()

        x = coords
        g = metric
        n = len(x)
        g_inv = g.inv()
        for mu in range(n):
            for alpha in range(n):
                for beta in range(n):
                    gamma = 0
                    for nu in range(n):
                        gamma += g_inv[mu, nu] * (sp.diff(g[alpha, nu], x[beta]) +
                                                  sp.diff(g[nu, beta], x[alpha]) -
                                                  sp.diff(g[alpha, beta], x[nu])
                                                  )
                    if gamma != 0:
                        self[(mu, alpha, beta)] = gamma / 2
```

and move/adapt implementation from Manifold class.
note: self[...]

Note the power of TDD!
